In [9]:
from functions.MAE import *
import pickle
import numpy as np
from torch import optim
import torch
import copy
from functions.benchmark import *
n_tasks = 4
combinations = pickle.load(open(f"C:\LocalData\Code\Thesis\evaluation_data\Test_combinations_{n_tasks}.pkl", "rb"))
test_data = pickle.load(open(f"evaluation_data\Test_data_{n_tasks}_100.pkl","rb"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_loss_function= torch.nn.MSELoss
optimizer = torch.optim.RAdam
train_loss_function = torch.nn.SmoothL1Loss
cols = [str(i) for i in np.arange(0,300,1)]
from functions.evaluation import evaluate_on_dataset

In [10]:
class Vanilla_MAE(nn.Module):
    def __init__(self, encoding_size, sequence_length, combinations, n_hidden_layers, hidden_size:int = -1, n_variable_layers:int = -1):
        super(Vanilla_MAE, self).__init__()                                     
        self.encoding_size = encoding_size
        self.sequence_length = sequence_length                                  
        self.combinations = combinations
        self.n_hidden_layers = n_hidden_layers
        self.hidden_size = hidden_size
        self.n_variable_layers = n_variable_layers
        self.task_IDs = []
        if self.n_hidden_layers == 0:
            self.fixed = nn.Sequential(nn.Linear(self.sequence_length, self.encoding_size),
                                       nn.ReLU())                               
            self.variable = nn.ModuleDict()                                     
            for shifting_type in combinations:                                  
                for transformer_ID in combinations[shifting_type]:
                    combination_ID = combination_to_id(shifting_type,transformer_ID, self.combinations)
                    self.task_IDs.append(int(combination_ID))
                    self.variable[str(combination_ID)] = nn.Linear(self.encoding_size, self.sequence_length)   
        elif self.n_hidden_layers == 1:
            if hidden_size == -1:
                raise ValueError("Choose a hidden size")
            elif hidden_size < 1:
                raise ValueError("Choose a positive hidden size")
            if self.n_variable_layers == 1:
                self.fixed = nn.Sequential(nn.Linear(self.sequence_length, self.hidden_size), 
                                        nn.ReLU(),
                                        nn.Linear(self.hidden_size, self.encoding_size),
                                        nn.ReLU(),
                                        nn.Linear(self.encoding_size, self.hidden_size),
                                        nn.ReLU())
                self.variable = nn.ModuleDict()                                         
                for shifting_type in combinations:                              
                    for transformer_ID in combinations[shifting_type]:
                        combination_ID = combination_to_id(shifting_type,transformer_ID, combinations) 
                        self.variable[str(combination_ID)] = nn.Linear(self.hidden_size, self.sequence_length)
            elif self.n_variable_layers == 2:
                self.fixed = nn.Sequential(nn.Linear(self.sequence_length, self.hidden_size),
                                        nn.ReLU(),
                                        nn.Linear(self.hidden_size, self.encoding_size),
                                        nn.ReLU())
                self.variable = nn.ModuleDict()                                     
                for shifting_type in combinations:                                  
                    for transformer_ID in combinations[shifting_type]:
                        combination_ID = combination_to_id(shifting_type,transformer_ID, combinations)
                        self.variable[str(combination_ID)] = nn.Sequential(nn.Linear(self.encoding_size, self.hidden_size),
                                                                    nn.ReLU(),
                                                                    nn.Linear(self.hidden_size, self.sequence_length)) 
            else:
                raise NotImplementedError("Please choose between 1 or 2 variable layers.")
        else:
            raise NotImplementedError("Please choose between 0 or 1 hidden layers.")

    def forward(self, x):
        x_id = x[:,self.sequence_length]
        sort = torch.argsort(x_id)
        reversesort = torch.argsort(sort)
        x = x[sort]
        y = []
        ids = torch.unique(x[:,self.sequence_length],sorted=False)
        for i in ids:
            x_i = x[x[:,self.sequence_length] == i]
            if x_i.shape[0] != 0:
                combination_ID = str(int(x_i[0,self.sequence_length].item()))
                x_i = x_i[:,:self.sequence_length]                           
                x_i = self.fixed(x_i)                                  
                x_i = self.variable[combination_ID](x_i)                         
                y.append(x_i)
        x = torch.concat(y,0)
        x = x[reversesort]
        return x
    
    def add_task(self, task_ID, shifting_type, transformer_ID, num_epochs, instances:int = 987654321, lr_schedule:bool = False, schedule_milestones = [60,90], convergence_plot:bool = True, return_training_loss:bool = True, path="C:\LocalData\Data\Completely_preprocessed_data"):
        if task_ID in self.task_IDs:
            raise ValueError("Task ID already used, please choose an unused task ID")
        if self.n_hidden_layers == 0:                                  
            self.variable[str(task_ID)] = nn.Linear(self.encoding_size, self.sequence_length)   
        elif self.n_hidden_layers == 1:
            if self.n_variable_layers == 1:
                self.variable[str(task_ID)] = nn.Linear(self.hidden_size, self.sequence_length)
            elif self.n_variable_layers == 2:
                self.variable[str(task_ID)] = nn.Sequential(nn.Linear(self.encoding_size, self.hidden_size),
                                                                    nn.ReLU(),
                                                                    nn.Linear(self.hidden_size, self.sequence_length))
        ds = create_dataset({shifting_type : {transformer_ID : task_ID}}, [str(i) for i in np.arange(0,300,1)], path)
        if instances != 987654321:
            ds.df_torque = ds.df_torque.sample(instances, replace=True)
        dl = torch.utils.data.DataLoader(ds, 64, True)
        for param in self.fixed.parameters():
            param.requires_grad = False
        loss_function = nn.SmoothL1Loss()
        optimizer = optim.Adam(self.variable.parameters(), 0.0001)
        if lr_schedule:
            scheduler = MultiStepLR(optimizer, milestones=schedule_milestones, gamma=0.1)
        losses = []
        for epoch in tqdm(range(num_epochs),leave=None):
            last_losses = []
            for data in dl:
                data= data.to(device=device)                                        #Bring the data onto the device
                reconstruction = self.forward(data)                                        #Execute the model
                loss = loss_function(reconstruction, data[:,:self.sequence_length])      #Compute the loss
                optimizer.zero_grad()                                               #Set the gradients to 0 for each batch
                loss.backward()                                                     #Compute gradients using backpropagation
                optimizer.step()                                                    #Execute parameter updates
                losses.append(loss.data)
                last_losses.append(loss.data)
            if lr_schedule:
                scheduler.step()
        if convergence_plot:                                                        #Optionally plot the convergence over the iterations
            plt.xlabel("Batch")
            plt.ylabel("Loss")
            plt.ylim(top = 20)
            plt.plot(torch.Tensor(losses).cpu(), linewidth = 0.2, color = "black")
            plt.show()
        self.task_IDs.append(task_ID)
        if return_training_loss:
            return torch.mean(torch.Tensor(losses).cpu())

In [11]:
class TCN_MAE(nn.Module):
    def __init__(self, sequence_length, combinations, n_channels:int, n_channels_reduced:int, n_encoding_channels:int, n_variable_layers:int, device, kernel_size:int = 5, pool_rate:int = 2, num_layers:int = 6):
        super(TCN_MAE, self).__init__()
        self.sequence_length = sequence_length
        self.combinations = combinations
        self.n_channels = n_channels
        self.n_channels_reduced = n_channels_reduced
        self.n_encoding_channels = n_encoding_channels
        self.pool_rate = pool_rate
        self.n_variable_layers = n_variable_layers
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.device = device
        self.task_IDs = []
        self.encoder = nn.ModuleDict()
        for i in range(num_layers):
            if i == 0:
                self.encoder[str(i)] = nn.Sequential(nn.Conv1d(1, self.n_channels, self.kernel_size,padding = 'same', dilation = 2**i),
                                                     nn.ReLU(),
                                                     nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                     nn.ReLU())
            else:
                self.encoder[str(i)] = nn.Sequential(nn.Conv1d(self.n_channels_reduced, self.n_channels, self.kernel_size,padding = 'same', dilation = 2**i),
                                                     nn.ReLU(),
                                                     nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                     nn.ReLU())
        self.encoder["compressor"] = nn.Sequential(nn.Conv1d(self.n_channels_reduced*self.num_layers, self.n_encoding_channels, 1),
                                                   nn.AvgPool1d(self.pool_rate))
        self.decoder_fixed = nn.ModuleDict()
        for i in range(self.num_layers - self.n_variable_layers):
            if i == 0:
                self.decoder_fixed[str(i)] = nn.Sequential(nn.Conv1d(self.n_encoding_channels, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                           nn.ReLU(),
                                                           nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                           nn.ReLU())
            else:
                self.decoder_fixed[str(i)] = nn.Sequential(nn.Conv1d(self.n_channels_reduced, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                           nn.ReLU(),
                                                           nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                           nn.ReLU())
        self.decoder_variable = nn.ModuleDict()
        for shifting_type in combinations:                                      
            for transformer_ID in combinations[shifting_type]:
                combination_ID = combination_to_id(shifting_type,transformer_ID, combinations) 
                self.task_IDs.append(str(combination_ID))
                self.decoder_variable[str(combination_ID)] = nn.ModuleDict()
                for i in range(self.num_layers - self.n_variable_layers, self.num_layers):
                    if i == 0:
                        self.decoder_variable[str(combination_ID)][str(i)] = nn.Sequential(nn.Conv1d(self.n_encoding_channels, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                                                        nn.ReLU(),
                                                                                        nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                                                        nn.ReLU())
                    else:
                        self.decoder_variable[str(combination_ID)][str(i)] = nn.Sequential(nn.Conv1d(self.n_channels_reduced, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                                                        nn.ReLU(),
                                                                                        nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                                                        nn.ReLU())
                self.decoder_variable[str(combination_ID)]["compressor"] = nn.Conv1d(self.n_channels_reduced*self.num_layers, 1, 1)
        
    def forward(self, x):
        y = []
        ids = torch.unique(x[:,self.sequence_length],sorted=False)
        for i in ids:
            x_i = x[x[:,self.sequence_length] == i]
            if x_i.shape[0] != 0:
                combination_ID = str(int(x_i[0,self.sequence_length].item()))    #Read the combination ID of first element of the batch
                x_i = x_i[:,:self.sequence_length] 
                x_i = x_i.unsqueeze(1)
                encoding = torch.empty(0).to(self.device)
                for j in range(self.num_layers):
                    x_i = self.encoder[str(j)](x_i)
                    encoding = torch.cat((encoding, x_i), dim = 1)
                encoding = self.encoder["compressor"](encoding)
                x_i = F.interpolate(encoding, self.sequence_length)
                decoding = torch.empty(0).to(self.device)
                for j in range(self.num_layers - self.n_variable_layers):
                    x_i = self.decoder_fixed[str(j)](x_i)
                    decoding = torch.cat((decoding, x_i), dim = 1)
                for j in range(self.num_layers - self.n_variable_layers, self.num_layers):
                    x_i = self.decoder_variable[str(combination_ID)][str(j)](x_i)
                    decoding = torch.cat((decoding, x_i), dim = 1)
                x_i = self.decoder_variable[str(combination_ID)]["compressor"](decoding)
                y.append(x_i)
        x = torch.concat(y,0)
        x = x.squeeze(1)
        return x
    
    def add_task(self, task_ID, shifting_type, transformer_ID, num_epochs, instances:int = 987654321, lr_schedule:bool = False, schedule_milestones = [60,90], convergence_plot:bool = True, return_training_loss:bool = True, path="C:\LocalData\Data\Completely_preprocessed_data"):
        if task_ID in self.task_IDs:
            raise ValueError("Task ID already used, please choose an unused task ID")
        self.decoder_variable[str(task_ID)] = nn.ModuleDict()
        for i in range(self.num_layers - self.n_variable_layers, self.num_layers):
            if i == 0:
                self.decoder_variable[str(task_ID)][str(i)] = nn.Sequential(nn.Conv1d(self.n_encoding_channels, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                                                nn.ReLU(),
                                                                                nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                                                nn.ReLU())
            else:
                self.decoder_variable[str(task_ID)][str(i)] = nn.Sequential(nn.Conv1d(self.n_channels_reduced, self.n_channels, self.kernel_size, dilation=2**(self.num_layers-1-i), padding = 'same'),
                                                                                nn.ReLU(),
                                                                                nn.Conv1d(self.n_channels, self.n_channels_reduced, 1),
                                                                                nn.ReLU())
                self.decoder_variable[str(task_ID)]["compressor"] = nn.Conv1d(self.n_channels_reduced*self.num_layers, 1, 1)        
        ds = create_dataset({shifting_type : {transformer_ID : task_ID}}, [str(i) for i in np.arange(0,300,1)], path)
        if instances != 987654321:
            ds.df_torque = ds.df_torque.sample(instances, replace=True)
        dl = torch.utils.data.DataLoader(ds, 64, True)
        for param in self.fixed.parameters():
            param.requires_grad = False
        loss_function = nn.SmoothL1Loss()
        optimizer = optim.Adam(self.variable.parameters(), 0.0001)
        if lr_schedule:
            scheduler = MultiStepLR(optimizer, milestones=schedule_milestones, gamma=0.1)
        losses = []
        for epoch in tqdm(range(num_epochs),leave=None):
            last_losses = []
            for data in dl:
                data= data.to(device=device)                                        #Bring the data onto the device
                reconstruction = self.forward(data)                                        #Execute the model
                loss = loss_function(reconstruction, data[:,:self.sequence_length])      #Compute the loss
                optimizer.zero_grad()                                               #Set the gradients to 0 for each batch
                loss.backward()                                                     #Compute gradients using backpropagation
                optimizer.step()                                                    #Execute parameter updates
                losses.append(loss.data)
                last_losses.append(loss.data)
            if lr_schedule:
                scheduler.step()
        if convergence_plot:                                                        #Optionally plot the convergence over the iterations
            plt.xlabel("Batch")
            plt.ylabel("Loss")
            plt.ylim(top = 20)
            plt.plot(torch.Tensor(losses).cpu(), linewidth = 0.2, color = "black")
            plt.show()
        self.task_IDs.append(task_ID)
        if return_training_loss:
            return torch.mean(torch.Tensor(losses).cpu())

In [16]:
class LSTM_MAE(nn.Module):
    def __init__(self, encoding_size, sequence_length, combinations, device, teacher_forcing:bool = True):
        super(LSTM_MAE, self).__init__()                        
        self.encoding_size = encoding_size
        self.sequence_length = sequence_length
        self.combinations = combinations
        self.device = device
        self.task_IDs = []
        self.fixed = nn.LSTM(input_size = 1,                                    #Create the fixed encoder network
                             hidden_size = self.encoding_size,
                             batch_first = True)
        self.variableLSTMs = nn.ModuleDict()                                    #Create the dictionary for the variable decoder LSTM networks
        self.variableLinears = nn.ModuleDict()                                  #Create the dictionary for the variable linear layer for the reconstruction from LSTM outputs, as done in Malhotra et al. (2016)
        self.teacher_forcing = teacher_forcing                                  #Optionally, use teacher forcing
        for shifting_type in self.combinations:                                 
            for transformer_ID in self.combinations[shifting_type]:
                combination_ID = combination_to_id(shifting_type, transformer_ID, self.combinations)
                self.task_IDs.append(int(combination_ID))
                self.variableLSTMs[str(combination_ID)] = nn.LSTM(input_size = 1, hidden_size = self.encoding_size, batch_first = False)
                self.variableLinears[str(combination_ID)] = nn.Linear(self.encoding_size, 1)

    def forward(self, x):
        x_id = x[:,self.sequence_length]                                        #Retrieve the column containing the combination ids
        sort = torch.argsort(x_id)                                              #Create a sorting element that sorts the observations according to their combination id
        reversesort = torch.argsort(sort)                                       #Create an inverse sorting element to undo the sorting at the end of the forward pass
        x = x[sort]                                                             #Execute the sorting
        y = []                                                                  #Create a list to save the prediction results for each id
        ids = torch.unique(x[:,self.sequence_length], sorted = False)           #Create a list containing the ids in the batch
        for i in ids:                                                           #Iterate over all these ids
            x_i = x[x[:,self.sequence_length] == i]                             #Get the data from only this id
            if x_i.shape[0] != 0:
                combination_ID = str(int(i))                                    #Retrieve the combination ID
                x_i = x_i[:,:self.sequence_length]                              #Discard the column containing the ids
                x_i = x_i.unsqueeze(2)                                          #Create dimension 2: input size = 1
                if self.teacher_forcing:
                    transposed_x_i = torch.transpose(x_i, 0, 1)                 #Transpose the data such that it can be used as input for the batch_first = False decoder when using teacer forcing
                _ , (h_t, c_t) = self.fixed(x_i)                                #Run the data through the encoder and retrieve the encoding
                complete_output = torch.empty(0).to(self.device)                     #Initialize a tensor to save the decoder output
                last_output = self.variableLinears[combination_ID](h_t)         #Initialize 0-th output directly from the encoding
                for step in range(self.sequence_length):                        #Iterate one step at a time such that the output of the previous step can be used as input for the next
                    last_output, (h_t, c_t) = self.variableLSTMs[combination_ID](last_output, (h_t, c_t)) #Compute the output of LSTM cell for this step
                    last_output = self.variableLinears[combination_ID](last_output) #Compute the actual output from the LSTM output for this step
                    complete_output = torch.cat((complete_output, last_output.squeeze(2)), dim = 0) #Add this output to the tensor that saves the decoder output
                    if self.teacher_forcing:
                        if self.training:                                       #When teacher forcing is enabled during the training phase, use the actual result as input for the next step
                            last_output = transposed_x_i[-(step+1),:,:].unsqueeze(0)
                y.append(complete_output)                                       #Add the complete output for this id to the list
        x = torch.cat(y,1)                                                      #Concatenate the results for all ids
        x = torch.transpose(x, 0, 1)                                            #Transpose the result to bring it in the same format as the input
        x = x[reversesort]                                                      #Undo the sorting that was done at the start
        x = torch.flip(x, [1])                                                  #Reconstruct the sequence in reverse order, as done in Malhotra et al. (2016)
        return x
    
    def add_task(self, task_ID, shifting_type, transformer_ID, num_epochs, instances:int = 987654321, lr_schedule:bool = False, schedule_milestones = [60,90], convergence_plot:bool = True, return_training_loss:bool = True, path="C:\LocalData\Data\Completely_preprocessed_data"):
        if task_ID in self.task_IDs:
            raise ValueError("Task ID already used, please choose an unused task ID")
        self.variableLSTMs[str(task_ID)] = nn.LSTM(input_size = 1, hidden_size = self.encoding_size, batch_first = False)
        self.variableLinears[str(task_ID)] = nn.Linear(self.encoding_size, 1)
        ds = create_dataset({shifting_type : {transformer_ID : task_ID}}, [str(i) for i in np.arange(0,300,1)], path)
        if instances != 987654321:
            ds.df_torque = ds.df_torque.sample(instances, replace=True)
        dl = torch.utils.data.DataLoader(ds, 64, True)
        for param in self.fixed.parameters():
            param.requires_grad = False
        loss_function = nn.SmoothL1Loss()
        optimizer = optim.Adam(self.variable.parameters(), 0.0001)
        if lr_schedule:
            scheduler = MultiStepLR(optimizer, milestones=schedule_milestones, gamma=0.1)
        losses = []
        for epoch in tqdm(range(num_epochs),leave=None):
            last_losses = []
            for data in dl:
                data= data.to(device=device)                                        #Bring the data onto the device
                reconstruction = self.forward(data)                                        #Execute the model
                loss = loss_function(reconstruction, data[:,:self.sequence_length])      #Compute the loss
                optimizer.zero_grad()                                               #Set the gradients to 0 for each batch
                loss.backward()                                                     #Compute gradients using backpropagation
                optimizer.step()                                                    #Execute parameter updates
                losses.append(loss.data)
                last_losses.append(loss.data)
            if lr_schedule:
                scheduler.step()
        if convergence_plot:                                                        #Optionally plot the convergence over the iterations
            plt.xlabel("Batch")
            plt.ylabel("Loss")
            plt.ylim(top = 20)
            plt.plot(torch.Tensor(losses).cpu(), linewidth = 0.2, color = "black")
            plt.show()
        self.task_IDs.append(task_ID)
        if return_training_loss:
            return torch.mean(torch.Tensor(losses).cpu())

In [14]:
models = ["Vanilla MAE", "Vanilla AE", "TCN MAE", "TCN AE", "LSTM MAE", "LSTM AE"]
from functions.benchmark import *
def evaluate_task_addition(combinations, sample_sizes, model, path="C:\LocalData\Data\Completely_preprocessed_data"):
    resultss = []
    for shifting_type in combinations:
        for transformer_ID in combinations[shifting_type]:
            test_combinations = copy.deepcopy(combinations)
            test_combinations[shifting_type].pop(transformer_ID)
            task_ID = combinations[shifting_type][transformer_ID]
            if model == "Vanilla MAE":
                m = Vanilla_MAE(250, 300, test_combinations, 0).to(device)
            elif model == "TCN MAE":
                m = TCN_MAE(300, test_combinations, 128, 32, 8, 6, device, 8, 32, 6).to(device)
            elif model == "LSTM MAE":
                m = LSTM_MAE(250, 300, test_combinations, device).to(device)
            elif model == "Vanilla AE":
                m = create_Vanilla_AE_separate(250, 300, {shifting_type : {transformer_ID : task_ID}}, 0, device)
            elif model == "TCN AE":
                m = create_TCN_AE_separate(300, {shifting_type : {transformer_ID : task_ID}}, 128, 32, 8, device, 8, 16, 6)
            elif model == "LSTM AE":
                m = create_LSTM_AE_separate(250, 300, {shifting_type : {transformer_ID : task_ID}}, device)
            else:
                print(f"Model {model} is not implemented")
                break
            if model in ["Vanilla MAE", "TCN MAE", "LSTM MAE"]:
                ds = create_dataset(test_combinations, cols, path)
                if model in ["Vanilla MAE", "TCN MAE"]:
                    dl = create_dataloader(ds, 64, 25)
                else:
                    create_mixed_batch_dataloader(ds, 16)
                train_model(m, 100, dl, train_loss_function, optimizer, device, 300, 0.0001, convergence_plot=False)
            results = []
            for sample_size in sample_sizes:
                test_model = copy.deepcopy(m)
                if model in ["Vanilla MAE", "TCN MAE", "LSTM MAE"]:
                    test_model.add_task(task_ID, shifting_type, transformer_ID, 100, int(sample_size), convergence_plot=False)
                else:
                    ds = create_dataset({shifting_type : {transformer_ID : task_ID}}, cols, path)
                    ds.df_torque = ds.df_torque.sample(int(sample_size))
                    dl = torch.utils.data.DataLoader(ds, 64, True)
                    train_model(test_model, 100, dl, nn.MSELoss, optim.Adam, device, 300, 0.0001, convergence_plot=False)
                td = test_data[test_data['combination_ID']==task_ID]
                result = evaluate_on_dataset(test_model, td, torch.nn.MSELoss(), shared_threshold=False, show_roc_curve=False)
                results.append(result[0])
            plt.plot(sample_sizes, results, scalex='log', label = str(task_ID))
            resultss.append(results)
    plt.legend()
    plt.xscale('log')
    plt.show()
    total = []
    for i in range(len(resultss[0])):
        total.append(np.mean([r[i] for r in resultss]))
    plt.plot(sample_sizes, total, scalex='log', color = 'black')
    plt.xscale('log')
    plt.show()
    return total, resultss

In [ ]:
sample_sizes = np.logspace(0,3,50)
r = []
for m in models:
    r.append(evaluate_task_addition(combinations, sample_sizes, m))
pickle.dump(r, open(f"task addition results {n_tasks}.pkl", "wb"))

In [ ]:
plt.plot(sample_sizes, r[0][0], color = 'black', label = "MAE")
plt.plot(sample_sizes, r[1][0], color = 'red', label = 'AE')
plt.xscale("log")
plt.title("Vanilla")
plt.show()
plt.plot(sample_sizes, r[2][0], color = 'black', label = "MAE")
plt.plot(sample_sizes, r[3][0], color = 'red', label = 'AE')
plt.xscale("log")
plt.title("TCN")
plt.show()
plt.plot(sample_sizes, r[4][0], color = 'black', label = "MAE")
plt.plot(sample_sizes, r[5][0], color = 'red', label = 'AE')
plt.title("LSTM")
plt.xscale("log")
plt.show()